## **INTRODUÇÃO**

O objetivo deste notebook é implementar uma estratégia de Parada Antecipada (Early Stopping) em uma rede neural MLP. Para isso, vamos treinar uma MLP com o dataset didático "Iris", do seaborn. Os dados serão divididos em treino, validação e teste.

Para o Early Stopping, enquanto otimizamos os parâmetros da nossa MLP com os dados de treino, vamos comparando a perda do teste com a perda de validação. Esperamos que, de início, as duas perdas diminuem ao passar das épocas. Mas, enquanto a perda do treino deve sempre diminuir, é esperado que em alguma época a perda da validação aumente, e é aí que aplicamos um early stopping, evitando um overfitting!

---

## **AUTORES E CONTRIBUIÇÕES**

**Autores:**

* Caio Matheus Leão Dantas
* Rafael Anis Shaikhzadeh Santos

**Contribuições:** Ambos discutiram o problema juntos, Caio Matheus realizou a primeira versão do código e Rafael Anis revisou o código e fez alterações para a versão final.

---

## **CÓDIGO**

#### ***Bibliotecas Importadas***

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from graphviz import Digraph
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import copy

### Funções importadas do material de aula

#### FUNÇÃO GRAFO

In [ ]:
def _tracar(folha):
    """Função modificada da criada por Andrej Karpathy para construção de grafo.

    Referência: https://github.com/karpathy/micrograd

    """
    vertices = set()
    arestas = set()

    def construir(v):
        """Função recursiva para traçar o grafo."""
        if v not in vertices:
            vertices.add(v)
            for progenitor in v.progenitor:
                arestas.add((progenitor, v))
                construir(progenitor)

    construir(folha)

    return vertices, arestas


def plota_grafo(folha):
    """Função modificada da criada por Andrej Karpathy para construção de grafo.

    Referência: https://github.com/karpathy/micrograd

    """
    grafo = Digraph(format="svg", graph_attr={"rankdir": "LR"})
    vertices, arestas = _tracar(folha)

    for v in vertices:
        id_vertice = str(id(v))

        if hasattr(v, "rotulo") and (hasattr(v, "grad")):
            texto = "{ " + f"{v.rotulo} | data {v.data:.3f} | grad {v.grad:.3f}" + " }"

        elif hasattr(v, "rotulo"):
            texto = "{ " + f"{v.rotulo} | data {v.data:.3f}" + " }"

        else:
            texto = "{ " + f"data {v.data:.3f}" + " }"

        grafo.node(name=id_vertice, label=texto, shape="record")

        if v.operador_mae:
            grafo.node(name=id_vertice + v.operador_mae, label=v.operador_mae)
            grafo.edge(id_vertice + v.operador_mae, id_vertice)

    for vertice1, vertice2 in arestas:
        grafo.edge(str(id(vertice1)), str(id(vertice2)) + vertice2.operador_mae)

    return grafo

#### CLASSE DA REDE NEURAL

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets):
        super().__init__()

        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.Sigmoid(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.Sigmoid(),
            nn.Linear(neuronios_c2, num_targets),
        )

    def forward(self, x):
        x = self.camadas(x)
        return x

#### Tratando dataframe

Importando nosso DataFrame:

In [ ]:
df = sns.load_dataset("iris")
df = df.dropna()
df = df[df["species"] == "setosa"] #só analisaremos as plantas da espécie setosa
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


Definindo os nossos atributos e target:

In [ ]:
X = [
    "sepal_length",
    "sepal_width",
    "petal_length"
]
y = ["petal_width"]

df = df.reindex(X + y, axis=1)
df

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


#### Split de dados

Vamos fazer isso em dois passos, primeiro dividir em `teste` e `treino/validação`, e depois dividir o treino/validação em `treino` e `validação`.

In [ ]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1
TAMANHO_TREINO = 1 - TAMANHO_TESTE - TAMANHO_VALIDACAO
TAMANHO_TREINO, TAMANHO_VALIDACAO, TAMANHO_TESTE
SEMENTE_ALEATORIA = 8

Definimos os nossos DataFrames de teste e de treino/validação:

In [ ]:
indices = df.index
indices_treino_val, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino_val = df.loc[indices_treino_val]
df_teste = df.loc[indices_teste]

X_teste = df_teste.reindex(X, axis=1).values
y_teste = df_teste.reindex(y, axis=1).values

In [ ]:
df_treino_val

,sepal_length,sepal_width,petal_length,petal_width
22,4.6,3.6,1.0,0.2
33,5.5,4.2,1.4,0.2
0,5.1,3.5,1.4,0.2
17,5.1,3.5,1.4,0.3
45,4.8,3.0,1.4,0.3
39,5.1,3.4,1.5,0.2
38,4.4,3.0,1.3,0.2
16,5.4,3.9,1.3,0.4
12,4.8,3.0,1.4,0.1
11,4.8,3.4,1.6,0.2


In [ ]:
df_teste

,sepal_length,sepal_width,petal_length,petal_width
44,5.1,3.8,1.9,0.4
49,5.0,3.3,1.4,0.2
36,5.5,3.5,1.3,0.2
23,5.1,3.3,1.7,0.5
48,5.3,3.7,1.5,0.2


E definimos, também, os nossos DataFrames de treino e validação:

In [ ]:
indices = df_treino_val.index
indices_treino, indices_val = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_val = df.loc[indices_val]

X_treino = df_treino.reindex(X, axis=1).values
y_treino = df_treino.reindex(y, axis=1).values

X_val = df_val.reindex(X, axis=1).values
y_val = df_val.reindex(y, axis=1).values

In [ ]:
df_treino

,sepal_length,sepal_width,petal_length,petal_width
37,4.9,3.6,1.4,0.1
9,4.9,3.1,1.5,0.1
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4
43,5.0,3.5,1.6,0.6
28,5.2,3.4,1.4,0.2
32,5.2,4.1,1.5,0.1
33,5.5,4.2,1.4,0.2
27,5.2,3.5,1.5,0.2
41,4.5,2.3,1.3,0.3


In [ ]:
df_val

,sepal_length,sepal_width,petal_length,petal_width
8,4.4,2.9,1.4,0.2
3,4.6,3.1,1.5,0.2
46,5.1,3.8,1.6,0.2
20,5.4,3.4,1.7,0.2
42,4.4,3.2,1.3,0.2


#### Normalização e transformação dos vetores em tensores

In [ ]:
x_scaler = StandardScaler()
x_scaler.fit(X_treino)

y_scaler = StandardScaler()
y_scaler.fit(y_treino)

X_treino = x_scaler.transform(X_treino)
y_treino = y_scaler.transform(y_treino)

X_val = x_scaler.transform(X_val)
y_val = y_scaler.transform(y_val)

X_teste = x_scaler.transform(X_teste)
y_teste = y_scaler.transform(y_teste)

Utilizamos a forma de tensores para os nossos dados e a utilização mais funcional deles na nossa rede:

In [ ]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

#### Trabalhando com a MLP

Vamos finalmente definir a nossa MLP e usá-la:

In [ ]:
NUM_DADOS_DE_ENTRADA = 3
NUM_DADOS_DE_SAIDA = 1
NEURONIOS_C1 = 3
NEURONIOS_C2 = 2

minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA
)

Prevemos, portanto, o nosso y baseado na rede que definimos:

In [ ]:
y_prev = minha_mlp(X_treino)
y_prev

tensor([[0.4959],
        [0.4939],
        [0.5227],
        [0.4789],
        [0.4802],
        [0.4992],
        [0.4813],
        [0.4897],
        [0.4895],
        [0.5162],
        [0.4702],
        [0.4816],
        [0.4834],
        [0.5153],
        [0.4865],
        [0.5016],
        [0.5054],
        [0.5030],
        [0.4851],
        [0.5212],
        [0.4579],
        [0.5092],
        [0.4913],
        [0.4902],
        [0.4874],
        [0.4977],
        [0.4939],
        [0.5080],
        [0.4810],
        [0.4851],
        [0.5084],
        [0.5033],
        [0.4962],
        [0.4870],
        [0.5030],
        [0.4975],
        [0.4905],
        [0.4678],
        [0.5000],
        [0.4977]], grad_fn=<AddmmBackward0>)

Podemos agora buscar otimizá-la, de modo a definir uma taxa de aprendizado, um otimizador (que nesse caso será o de `descida do gradiente estocástico (SGD)` da própria biblioteca *Torch*) e a função de perda, que aqui será o erro quadrático médio (MSE).

In [ ]:
TAXA_DE_APRENDIZADO = 0.001

otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)
fn_perda = nn.MSELoss()

Perfeito, podemos seguir pra melhor parte. Vamos treinar nossa MLP com um Early Stopping! Esse treino será feito com loops em épocas, exatamente como sempre vimos. Calculamos a perda do treino e usamos essa para otimizar os parâmetros.

Mas aqui, teremos um passo a mais, para cada época também vamos analisar a perda da validação, nos mantendo alerta para se sua perda também está diminuindo. Se em alguma época a perda subir, está na hora de parar nosso treino, evitando um overfitting.

Porém, ao invés de parar instantaneamente, vamos ser pacientes! Vamos rodar mais algumas épocas, e ver se perda cai de novo e alcança um novo melhor valor de perda. Se mesmo, depois de sermos pacientes nada melhora, aí sim! Paramos o nosso código, tendo guardado os parâmetros da melhor época.

DEFININDO O VALOR DE PACIÊNCIA

In [ ]:
patience = 30
melhor_val_loss = float('inf')
contador_paciencia = 0
melhores_pesos = None

In [ ]:
N_EPOCAS = 10000

minha_mlp.train()

for epoca in range(N_EPOCAS):
    minha_mlp = minha_mlp.double()
    X_treino = X_treino.double()
    y_treino = y_treino.double()
    y_pred = minha_mlp(X_treino)
    otimizador.zero_grad()
    loss_treino = fn_perda(y_treino, y_pred)
    loss_treino.backward()
    otimizador.step()

    #Validação
    minha_mlp.eval()
    with torch.no_grad():
      X_val = X_val.double()
      y_val = y_val.double()
      y_val_pred = minha_mlp(X_val)
      loss_val = fn_perda(y_val, y_val_pred)

    print(epoca, "- Perda do treino: ", loss_treino.item(), "/ Perda da validação: ", loss_val.item())

    #Early Stopping
    if loss_val.item() < melhor_val_loss - 1e-3:
        melhor_val_loss = loss_val.item()
        contador_paciencia = 0
        melhores_pesos = copy.deepcopy(minha_mlp.state_dict())
    else:
        contador_paciencia += 1
        if contador_paciencia >= patience:
            print(f"Pausa na época {epoca} pois val_loss não melhorou por {patience} épocas), o melhor valor de perda seria de {melhor_val_loss}, na epoca {epoca-patience}")
            break

0 - Perda do treino:  1.2523325595535606 / Perda da validação:  0.8443724088522165
1 - Perda do treino:  1.2508454789722125 / Perda da validação:  0.8416423485490376
2 - Perda do treino:  1.249367484630612 / Perda da validação:  0.8389252016646035
3 - Perda do treino:  1.2478985192085519 / Perda da validação:  0.8362208982307304
4 - Perda do treino:  1.2464385257659942 / Perda da validação:  0.833529368707673
5 - Perda do treino:  1.2449874477403524 / Perda da validação:  0.8308505439811732
6 - Perda do treino:  1.2435452289437965 / Perda da validação:  0.8281843553595344
7 - Perda do treino:  1.2421118135605762 / Perda da validação:  0.8255307345707157
8 - Perda do treino:  1.2406871461443694 / Perda da validação:  0.8228896137594509
9 - Perda do treino:  1.2392711716156473 / Perda da validação:  0.82026092548439
10 - Perda do treino:  1.2378638352590645 / Perda da validação:  0.8176446027152597
11 - Perda do treino:  1.2364650827208667 / Perda da validação:  0.8150405788300507
12 - P

---

## **CONCLUSÃO**

Podemos ver que a partir da época 1224, a perda da validação não melhorou, e portanto depois de 30 épocas (definida pela paciência) o treino parou. Assim, a estratégia de Early Stopping foi implementada com sucesso, com o objetivo de evitar overfitting.

Com esse notebook se aprendeu sobre Early Stopping e também sobre o conceito de paciência, algo novo para os dois autores.

---

## **REFERÊNCIAS**

**[1]** CASSAR, Daniel. Redes Neurais e Algoritmos Genéticos. 2025. Material de Aula.

**[2]** KASHYAP, Piyush. Early Stopping in Deep Learning: A Simple Guide to Prevent Overfitting. Medium. 2021. Disponível em: https://medium.com/@piyushkashyap045/early-stopping-in-deep-learning-a-simple-guide-to-prevent-overfitting-1073f56b493e.

**[3]** BROWNLEE, Jason. How to Stop Training Deep Neural Networks at the Right Time Using Early Stopping. Machine Learning Mastery. 2019. Disponível em: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/.

**[4]** OLAMENDY, Juan C. Real World ML: Early Stopping in Deep Learning - A Comprehensive Guide. Medium. 2023. Disponível em: https://medium.com/@juanc.olamendy/real-world-ml-early-stopping-in-deep-learning-a-comprehensive-guide-fabb1e69f8cc.